In [1]:
#defaul libraries
#https://docs.python.org/ja/
import os
import sys
import io

import math
import random
import pprint
import time
import datetime
import typing
import json
import glob
import requests
import warnings
import gc
from pprint import pprint
import re

import numpy as np #https://numpy.org/
import pandas as pd #https://pandas.pydata.org/
import sklearn #https://scikit-learn.org/stable/

import matplotlib.pyplot as plt #https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html
%matplotlib inline

import seaborn as sns
sns.set()

from tqdm import tqdm #https://tqdm.github.io/

import torch #https://pytorch.org/
import transformers #https://huggingface.co/transformers/

#import torchvision
#import torchtext


In [2]:
class CFG():
    
    data_path="../input/multinli-mismatched-open-evaluation/"#"./multinli_1.0/"
    debug=False
    seed=0

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    batch_size=8
    epochs=2
    learning_rate=2e-5
    kFold=1


    #高速化関連
    #https://qiita.com/sugulu_Ogawa_ISID/items/62f5f7adee083d96a587

    #GPU 遅くなるらしい↓
    torch.backends.cudnn.deterministic = True

    #イテレーションごとのnnの順伝搬および誤差関数の 計算手法がある程度一定であれば、torch.backends.cudnn.benchmark = Trueで GPU での計算が高速化
    torch.backends.cudnn.benchmark = False


def set_seed(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    #tf.random.set_seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(CFG.seed)

print(CFG.device)
if torch.cuda.is_available():
    print(torch.cuda.current_device())
    print(torch.cuda.get_device_name())

cuda:0
0
Tesla P100-PCIE-16GB


In [3]:
def color(string,fg='DEFAULT',bg='DEFAULT',fg_rgb=None,bg_rgb=None,style='END'):
    colors=['BLACK','RED','GREEN','YELLOW','BLUE','PURPLE','CYAN','WHITE','8','DEFAULT']
    styles=['END','BOLD','2','3','UNDERLINE','5','6','REVERSE','INVISIBLE','9']

    fg=f'\033[3{colors.index(fg)}m'
    bg=f'\033[4{colors.index(bg)}m'
    style=f'\033[0{styles.index(style)}m'

    if fg_rgb:fg=f"\033[38;2;{fg_rgb[0]};{fg_rgb[1]};{fg_rgb[2]}m"
    if bg_rgb:bg=f"\033[48;2;{bg_rgb[0]};{bg_rgb[1]};{bg_rgb[2]}m"

    return style+fg+bg+str(string)+'\033[0m'

In [4]:
print(color("color test","GREEN",style='BOLD'))
print(color("color test","RED","BLUE",style='UNDERLINE'))
print(color("color test","CYAN","WHITE",style='INVISIBLE'))
print(color("color test","CYAN",style='REVERSE'))
print(color("aaa",fg_rgb=(150,150,255)))
# print(color("bbb",fg_rgb=(0,250,50),bg_rgb=(0,0,255)))
# print(color("ccc"))
# print(color('style',bg="RED",fg='BLUE'))
# print('\033[07m'+"Aa")

color test
color test
color test
color test
aaa


In [5]:
import requests
import datetime
import json

class SlackNotifier():
    
    def __init__(self,token,channel):
        self.token=token
        self.channel=channel
    
    def textblock(self,text):
        blocks=json.dumps(
            [#ヘッダー
                #現在時刻
                {
                    "type": "context",
                    "elements": [
                        {
                            "type": "plain_text",
                            "text": (datetime.datetime.utcnow() + datetime.timedelta(hours=9)).strftime('%Y年%m月%d日 %H:%M:%S'),
                            "emoji": True
                        }
                    ]
                },
                {"type": "divider"},
                {
                    "type": "section",
                    "text": {
                        "type": "mrkdwn",
                        "text": text
                    }
                }
            ],
        )
        return blocks
    
    def post(self,message):
        pass
        
    
    def update(self,timestamp,blocks):
        
        response=requests.post(
            "https://slack.com/api/chat.update",
            data={
                "token":self.token,
                "channel":self.channel,
                "ts":timestamp,
                #"blocks":
                'blocks':blocks,
            },
        )
        return response
    
token=""
channel=""
timestamp=""

slack=SlackNotifier(token,channel)

In [6]:
multi_nli_test_mismatched=pd.read_json(CFG.data_path+'multinli_0.9_test_mismatched_unlabeled.jsonl',orient='records',lines=True)


if CFG.debug:
    multi_nli_test_mismatched=multi_nli_test_mismatched.sample(frac=0.01).reset_index(drop=True)

In [7]:
multi_nli_test_mismatched
#multi_nli_test_mismatched[multi_nli_test_mismatched.gold_label=='-']#意見が割れてる

,annotator_labels,genre,gold_label,pairID,promptID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,"[hidden, hidden, hidden, hidden, hidden]",oup,hidden,0,0,"Even here, the channel perspective tells a som...","( ( Even here ) ( , ( ( the ( channel perspect...","(ROOT (S (ADVP (RB Even) (RB here)) (, ,) (NP ...",The story told by the channel is slightly diff...,( ( ( The story ) ( told ( by ( the channel ) ...,(ROOT (S (NP (NP (DT The) (NN story)) (VP (VBN...
1,"[hidden, hidden, hidden, hidden, hidden]",verbatim,hidden,1,1,My theory is that the r got put in there simp...,( ( My theory ) ( ( is ( that ( ( the r ) ( ( ...,(ROOT (S (NP (PRP$ My) (NN theory)) (VP (VBZ i...,I think the r is there just because people are...,( I ( ( think ( ( the r ) ( ( is there ) ( jus...,(ROOT (S (NP (PRP I)) (VP (VBP think) (SBAR (S...
2,"[hidden, hidden, hidden, hidden, hidden]",verbatim,hidden,2,2,To go on using in its former sense a word whos...,( ( To ( go ( on ( ( using ( in ( its ( former...,(ROOT (S (S (VP (TO To) (VP (VB go) (PP (IN on...,To go on using it in its former sense could be...,( ( To ( go ( on ( ( using it ) ( in ( its ( f...,(ROOT (NP (S (S (VP (TO To) (VP (VB go) (PP (I...
3,"[hidden, hidden, hidden, hidden, hidden]",nineeleven,hidden,3,3,"He alerted his supervisor, who assigned anothe...","( He ( ( alerted ( ( ( his supervisor ) , ) ( ...",(ROOT (S (NP (PRP He)) (VP (VBD alerted) (NP (...,The supervisor had another controller work wit...,( ( The supervisor ) ( ( had ( ( another ( con...,(ROOT (S (NP (DT The) (NN supervisor)) (VP (VB...
4,"[hidden, hidden, hidden, hidden, hidden]",letters,hidden,4,4,I have also enclosed a brief report on those a...,( I ( ( ( have also ) ( ( enclosed ( a ( brief...,(ROOT (S (NP (PRP I)) (VP (VBP have) (ADVP (RB...,Your annual contribution will have little impact.,( ( Your ( annual contribution ) ) ( ( will ( ...,(ROOT (S (NP (PRP$ Your) (JJ annual) (NN contr...
...,...,...,...,...,...,...,...,...,...,...,...
9842,"[hidden, hidden, hidden, hidden, hidden]",facetoface,hidden,9842,9842,Grandmom smoked Saratoga 120 cigarettes which ...,( Grandmom ( ( smoked ( ( Saratoga ( 120 cigar...,(ROOT (S (NP (NNP Grandmom)) (VP (VBD smoked) ...,Those really long Saratoga 120 cigarettes are ...,( ( Those ( ( really long ) ( Saratoga ( 120 c...,(ROOT (S (NP (DT Those) (ADJP (RB really) (JJ ...
9843,"[hidden, hidden, hidden, hidden, hidden]",oup,hidden,9843,9843,"But if order lead times are long, the buyer mu...",( But ( ( ( if order ) ( ( lead times ) ( are ...,(ROOT (S (CC But) (SBAR (IN if) (NN order) (S ...,Orders from China often take longer than expec...,( ( Orders ( from China ) ) ( often ( ( take (...,(ROOT (S (NP (NP (NNS Orders)) (PP (IN from) (...
9844,"[hidden, hidden, hidden, hidden, hidden]",nineeleven,hidden,9844,9844,"After NEADS learned of the hijacking at 10:07,...",( ( After ( NEADS ( learned ( of ( ( the hijac...,(ROOT (S (PP (IN After) (NP (NP (NNS NEADS)) (...,"NEADS learned of the hijacking at 10:07, befor...",( NEADS ( ( ( ( ( learned ( of ( the hijacking...,(ROOT (S (NP (NNS NEADS)) (VP (VBD learned) (P...
9845,"[hidden, hidden, hidden, hidden, hidden]",facetoface,hidden,9845,9845,"I said, And do you think you can help me?","( ( ( ( I said ) , ) And ) ( ( ( do you ) ( th...","(ROOT (S (S (NP (PRP I)) (VP (VBD said))) (, ,...","I asked, so do you think you can help me with ...","( ( ( ( I asked ) , ) so ) ( ( ( do you ) ( th...","(ROOT (S (S (NP (PRP I)) (VP (VBD asked))) (, ..."


In [8]:
tokenizer=transformers.BertTokenizer.from_pretrained("../input/bert-base-uncased")

In [9]:
multi_nli_test_mismatched['sentence1_len']=multi_nli_test_mismatched.sentence1.apply(tokenizer.encode).apply(len)
multi_nli_test_mismatched['sentence2_len']=multi_nli_test_mismatched.sentence2.apply(tokenizer.encode).apply(len)
max_len=max(multi_nli_test_mismatched['sentence1_len']+multi_nli_test_mismatched['sentence2_len'])+3
max_len#=448 #他のデータにもっと長いのあるかも

266

In [10]:
max_len=448

In [11]:
# from sklearn.model_selection import train_test_split
# multi_nli_dev_matched_val, multi_nli_dev_matched_test = sklearn.model_selection.train_test_split(
#     multi_nli_dev_matched,
#     test_size=0.8,
#     random_state=CFG.seed, 
#     stratify=multi_nli_dev_matched["gold_label"],
# )

# multi_nli_dev_mismatched_val, multi_nli_dev_mismatched_test = sklearn.model_selection.train_test_split(
#     multi_nli_dev_mismatched,
#     test_size=0.8,
#     random_state=CFG.seed, 
#     stratify=multi_nli_dev_mismatched["gold_label"],
# )

# multi_nli_train=multi_nli_train.reset_index(drop=True)

# multi_nli_dev_matched_val=multi_nli_dev_matched_val.reset_index(drop=True)
# multi_nli_dev_matched_test=multi_nli_dev_matched_test.reset_index(drop=True)

# multi_nli_dev_mismatched_val=multi_nli_dev_mismatched_val.reset_index(drop=True)
# multi_nli_dev_mismatched_test=multi_nli_dev_mismatched_test.reset_index(drop=True)

In [12]:
multi_nli_test_mismatched

,annotator_labels,genre,gold_label,pairID,promptID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse,sentence1_len,sentence2_len
0,"[hidden, hidden, hidden, hidden, hidden]",oup,hidden,0,0,"Even here, the channel perspective tells a som...","( ( Even here ) ( , ( ( the ( channel perspect...","(ROOT (S (ADVP (RB Even) (RB here)) (, ,) (NP ...",The story told by the channel is slightly diff...,( ( ( The story ) ( told ( by ( the channel ) ...,(ROOT (S (NP (NP (DT The) (NN story)) (VP (VBN...,14,12
1,"[hidden, hidden, hidden, hidden, hidden]",verbatim,hidden,1,1,My theory is that the r got put in there simp...,( ( My theory ) ( ( is ( that ( ( the r ) ( ( ...,(ROOT (S (NP (PRP$ My) (NN theory)) (VP (VBZ i...,I think the r is there just because people are...,( I ( ( think ( ( the r ) ( ( is there ) ( jus...,(ROOT (S (NP (PRP I)) (VP (VBP think) (SBAR (S...,35,16
2,"[hidden, hidden, hidden, hidden, hidden]",verbatim,hidden,2,2,To go on using in its former sense a word whos...,( ( To ( go ( on ( ( using ( in ( its ( former...,(ROOT (S (S (VP (TO To) (VP (VB go) (PP (IN on...,To go on using it in its former sense could be...,( ( To ( go ( on ( ( using it ) ( in ( its ( f...,(ROOT (NP (S (S (VP (TO To) (VP (VB go) (PP (I...,21,16
3,"[hidden, hidden, hidden, hidden, hidden]",nineeleven,hidden,3,3,"He alerted his supervisor, who assigned anothe...","( He ( ( alerted ( ( ( his supervisor ) , ) ( ...",(ROOT (S (NP (PRP He)) (VP (VBD alerted) (NP (...,The supervisor had another controller work wit...,( ( The supervisor ) ( ( had ( ( another ( con...,(ROOT (S (NP (DT The) (NN supervisor)) (VP (VB...,15,11
4,"[hidden, hidden, hidden, hidden, hidden]",letters,hidden,4,4,I have also enclosed a brief report on those a...,( I ( ( ( have also ) ( ( enclosed ( a ( brief...,(ROOT (S (NP (PRP I)) (VP (VBP have) (ADVP (RB...,Your annual contribution will have little impact.,( ( Your ( annual contribution ) ) ( ( will ( ...,(ROOT (S (NP (PRP$ Your) (JJ annual) (NN contr...,21,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9842,"[hidden, hidden, hidden, hidden, hidden]",facetoface,hidden,9842,9842,Grandmom smoked Saratoga 120 cigarettes which ...,( Grandmom ( ( smoked ( ( Saratoga ( 120 cigar...,(ROOT (S (NP (NNP Grandmom)) (VP (VBD smoked) ...,Those really long Saratoga 120 cigarettes are ...,( ( Those ( ( really long ) ( Saratoga ( 120 c...,(ROOT (S (NP (DT Those) (ADJP (RB really) (JJ ...,19,13
9843,"[hidden, hidden, hidden, hidden, hidden]",oup,hidden,9843,9843,"But if order lead times are long, the buyer mu...",( But ( ( ( if order ) ( ( lead times ) ( are ...,(ROOT (S (CC But) (SBAR (IN if) (NN order) (S ...,Orders from China often take longer than expec...,( ( Orders ( from China ) ) ( often ( ( take (...,(ROOT (S (NP (NP (NNS Orders)) (PP (IN from) (...,31,11
9844,"[hidden, hidden, hidden, hidden, hidden]",nineeleven,hidden,9844,9844,"After NEADS learned of the hijacking at 10:07,...",( ( After ( NEADS ( learned ( of ( ( the hijac...,(ROOT (S (PP (IN After) (NP (NP (NNS NEADS)) (...,"NEADS learned of the hijacking at 10:07, befor...",( NEADS ( ( ( ( ( learned ( of ( the hijacking...,(ROOT (S (NP (NNS NEADS)) (VP (VBD learned) (P...,71,19
9845,"[hidden, hidden, hidden, hidden, hidden]",facetoface,hidden,9845,9845,"I said, And do you think you can help me?","( ( ( ( I said ) , ) And ) ( ( ( do you ) ( th...","(ROOT (S (S (NP (PRP I)) (VP (VBD said))) (, ,...","I asked, so do you think you can help me with ...","( ( ( ( I asked ) , ) so ) ( ( ( do you ) ( th...","(ROOT (S (S (NP (PRP I)) (VP (VBD asked))) (, ...",14,17


In [13]:
data={
    'test':{
        'mismatched':multi_nli_test_mismatched,
    },
}

In [14]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self,sentences1,sentences2,targets=None,phase='train'):
        self.sentences1=sentences1
        self.sentences2=sentences2
        self.targets=targets
        self.phase=phase #train/val/test

    def __len__(self):
        return len(self.sentences1)

    def __getitem__(self,idx):
        
        enc=tokenizer(
            self.sentences1[idx],
            self.sentences2[idx],
            padding='max_length',
            max_length=max_len,
        )
        return {
            'input_ids':torch.tensor(enc['input_ids'],dtype=torch.long),
            'attention_mask':torch.tensor(enc['attention_mask'],dtype=torch.float),
            'token_type_ids':torch.tensor(enc['token_type_ids'],dtype=torch.long),
            #'targets':torch.tensor(self.targets[idx],dtype=torch.float)
        },torch.tensor(-1)
    #torch.tensor(['entailment','neutral','contradiction'].index(self.targets[idx]))
    #torch.tensor(list(map(lambda label: int(label==self.targets[idx]), ['entailment','neutral','contradiction'])))

In [15]:
def set_dataset():
    global dataset
    dataset={
        'test'  :{
            'mismatched':Dataset(
                data['test']['mismatched'].sentence1,
                data['test']['mismatched'].sentence2,
                data['test']['mismatched'].gold_label,
                'test'
            ),
        },
    }

set_dataset()

In [16]:
dataset['test']['mismatched'][2]

({'input_ids': tensor([  101,  2000,  2175,  2006,  2478,  1999,  2049,  2280,  3168,  1037,
           2773,  3005,  3574,  2038,  2904,  2003,  4675, 21572, 26638,  1012,
            102,  2000,  2175,  2006,  2478,  2009,  1999,  2049,  2280,  3168,
           2071,  2022,  2641,  2062,  6742,   102,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,    

In [17]:
def set_dataloader():
    global dataloader
    dataloader={
        'test':{
            'mismatched':torch.utils.data.DataLoader(
                dataset['test']['mismatched'],
                batch_size=CFG.batch_size,
                shuffle=False,
                num_workers=os.cpu_count(),
                pin_memory=True
            ),
        },
    }
set_dataloader()

In [18]:
bestmodel=torch.load("../input/argument-mining-graduation-thesis/bert_mnli_0.8428280773143438.pth")

In [19]:
bestmodel['bestscore']

0.8428280773143438

In [20]:
##create_token_type_ids_from_sequences
model = transformers.BertForSequenceClassification.from_pretrained("../input/bert-base-uncased",num_labels=3)
model.to(CFG.device)



Some weights of the model checkpoint at ../input/bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model chec

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [21]:
model.load_state_dict(bestmodel['state_dict'])

<All keys matched successfully>

In [22]:
# a = torch.randn(4, 3)
# a
# b=torch.tensor([1,2,1,2])

In [23]:
# torch.argmax(a,dim=1)==b

In [24]:
def score_fun(preds,y):
    return (torch.argmax(preds,dim=1)==y).detach().cpu().numpy()

In [25]:
class Trainer():

    
    def __init__(self):
        self.loss_fun=torch.nn.CrossEntropyLoss()
        self.score_fun=score_fun

        self.optimizer=torch.optim.AdamW(model.parameters(),lr=CFG.learning_rate,eps=1e-6)
        self.scheduler=torch.optim.lr_scheduler.LambdaLR(self.optimizer, lr_lambda = lambda epoch: 0.95 ** epoch, verbose=True)
        #scaler=scaler
        #self.initialize()
    
    def train_val_test(self,model,dataloader,phase):
    
        preds=[]
        losses=[]
        scores=[]
        
        model.train() if phase=='train' else model.eval()# モデルのモード
        
        # データローダーからミニバッチを取り出すループ
        
        for features,targets in tqdm(dataloader):
            
            # optimizerを初期化
            if phase=='train':self.optimizer.zero_grad()

            # 順伝搬（forward）計算
            with torch.set_grad_enabled(phase=='train'):
 
                input_ids      =features['input_ids'].to(CFG.device,non_blocking=True)
                attention_mask =features['attention_mask'].to(CFG.device,non_blocking=True)
                token_type_ids=features['token_type_ids'].to(CFG.device,non_blocking=True)
                targets        =targets.to(CFG.device,non_blocking=True)
                
                #with torch.cuda.amp.autocast():

                
                outputs = model(input_ids,attention_mask,token_type_ids)#,labels=targets)
                
#                 print(outputs)
#                 print(targets)

                    
                #if phase!='test':
        
#                 #loss = self.loss_fun(outputs, targets)  # 損失を計算
#                 loss=outputs.loss
#                 score = self.score_fun(outputs.logits, targets)  # 損失を計算
#                 #print(score)
#                 losses.append(loss.item())
#                 scores.extend(score)

                # 訓練時はバックプロパゲーション
                if phase == 'train':
                    loss.backward()
                    #loss_val.backward()
                    self.optimizer.step()

                    #scalerの場合
                    #scaler.scale(loss_val).backward() # ロスのバックワード
                    #scaler.step(optimizer) # オプティマイザーの更新
                    #scaler.update() # スケーラーの更新

                    #scheduler.step() # 学習率の更新 ここに入れるべき？　まだ使わない
                        
            #preds.extend(outputs.logits.detach().cpu().numpy())
            
            #print(outputs)
            
            #print(((list(map(lambda x: ['entailment','neutral','contradiction'][x], torch.argmax(outputs.logits,dim=1).detach().cpu().numpy())))))
            
            preds.extend(((list(map(lambda x: ['entailment','neutral','contradiction'][x], torch.argmax(outputs.logits,dim=1).detach().cpu().numpy())))))
            
            
            del features,targets,outputs
            #if phase!='test':del loss,score
            torch.cuda.empty_cache()
            gc.collect()
            
        return preds,losses,scores
    
    def cross_validation(self,model,dataloader):
        
        for fold in range(CFG.kFold):
            print('fold',fold)
            
            losses=[]
            scores=[]
            
            #self.initialize(CFG.seed,fold)
            #dataset,dataloader,model,optimizer,scheduler,scaler=initialize(CFG.seed,fold)
            
            bestscore=0
            
            tqdm_bar=io.StringIO()
            for epoch in tqdm(range(CFG.epochs),file=tqdm_bar,desc='BERT Multi-NLI'):
                print("epoch=",epoch)

                preds,_,score=self.train_val_test(model,dataloader['train'],'train')
                print(color("train score",bg='CYAN')+' :',color(np.mean(score),'CYAN'))
                
                #plt.scatter(dataset['train'].targets,preds,color='blue',s=5)
                
                #matched
                _,loss,score=self.train_val_test(model,dataloader['val']['matched'],'val')
                print(np.mean(loss))
                print(color("matched score",bg='BLUE')+' :',color(np.mean(score),'BLUE'))
                
                #mismatched
                _,loss,score=self.train_val_test(model,dataloader['val']['mismatched'],'val')
                print(np.mean(loss))
                print(color("mismatched score",bg='RED')+' :',color(np.mean(score),'RED'))
                
                scores.append(np.mean(score))
                losses.append(np.mean(loss))
                
                self.optimizer.step()
                
                #print(scores[-1])

                if bestscore < scores[-1]:
                    bestscore = scores[-1]
                    print(color("BEST SCORE",bg='YELLOW')+' :',color(bestscore,'YELLOW'))

                    bestmodel={
                        'state_dict': model.state_dict(),
                        'optimizer_dict': self.optimizer.state_dict(),
                        'bestscore':bestscore,
                        'seed':CFG.seed
                    }

                #print(preds)

                slack.update(
                    timestamp,
                    slack.textblock(
                        tqdm_bar.getvalue().split('\r')[-1]+
                        f"loss={losses[-1]:.2f} score={scores[-1]:.2f} "
                    )
                )
                
            pd.Series(losses).plot()
            plt.show()
            pd.Series(scores).plot()
            plt.show()

            torch.save(bestmodel,"bert_mnli_"+str(bestscore)+".pth")

#             plt.plot(losses['train'],color='blue')
#             plt.plot(losses['valid'],color='red')
#             plt.show()
    
            
trainer=Trainer()

Adjusting learning rate of group 0 to 2.0000e-05.


In [26]:
# trainer.cross_validation(model,dataloader)


In [27]:
# preds,losses,scores=trainer.train_val_test(model,dataloader['test']['matched'],'test')
# print(np.mean(scores))

In [28]:
preds,losses,scores=trainer.train_val_test(model,dataloader['test']['mismatched'],'test')
print(np.mean(scores))

100%|██████████| 1231/1231 [05:00<00:00,  4.09it/s]

nan



/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [29]:
#preds

In [30]:
# slack.update(
#     timestamp,
#     slack.textblock(
#         f"train completed. loss={np.mean(losses):.2f} score={np.mean(scores):.2f} "
#     )
# )

In [31]:
#torch.save(model.state_dict(), 'bert_multi_nli')

In [32]:
submission=pd.DataFrame(index=data['test']['mismatched'].pairID,columns=['gold_label'])

#submission.index=
submission.gold_label=preds

submission.head()

,gold_label
pairID,
0,entailment
1,neutral
2,neutral
3,entailment
4,neutral


In [33]:
submission.to_csv("submission.csv",index='pairID')